In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
cd /content/drive/MyDrive/DLproject/DLproject

/content/drive/MyDrive/DLproject/DLproject


In [9]:
!pip install ipywidgets==7.7.1
!git config --global user.email 'sus5467@naver.com'
!git config --global user.name 'xuio-0528'

In [15]:
import huggingface_hub

In [16]:
huggingface_hub.login()

In [22]:
! git add .

fatal: not a git repository (or any of the parent directories): .git


In [23]:
! git commit -m "llama-7b 돌아가는 것 확인"

fatal: not a git repository (or any of the parent directories): .git


In [21]:
! git push

fatal: not a git repository (or any of the parent directories): .git


## 개요

어떤 유형의 프로젝트를 진행할 것인가?
### 후보군
1. prompt tuning
2. LoRA
3. small model 학습

### 장단점
#### 1. prompt tuning
- prompt tuning의 경우 오랜 학습이 필요하지 않음
- 어떤 부분을 차용해서 해볼 것인지에 대한 고려가 필요하다.

#### 2. LoRA
- 오랜 학습이 필요하지는 않음
- LoRA로 무엇을 학습시킬 것인지 고안하는 것이 중요
- 학습이 잘 되지 않을 수 있다는 위험성이 존재


#### 3. small model 학습
- 성능 상승이 거의 일어나기 어렵다.
- 별로 매력적이지 않음

#### 번외
- 1번 2번 두 가지 모두 이용을 해보는 것도 좋은 선택지가 될 것 같다.

### 다룰 모델들은?
1. 코드 생성 모델
2. reasoning model

위의 두 모델 같은 경우에는 아무래도 정확도가 어느 정도 담보가 되어 있는 모델들이 좋다.

코드 생성 모델의 경우 phi-1.5를 사용할 것 같다.
reasoning model의 경우 llama를 사용하는 것이 좋을 것 같다.

그렇다면 LoRA로 무엇을 가르쳐볼까?

일반적으로 우리는 prompt를 학습하거나 freeze시키는 등을 통해 embedding에서의 수정을 해보고자 한다.

혹은 하나의 prompt만을 동일하게 사용하려는 것을 보인다.
예를 들어, let's think about step by step 등

그렇다면 prompt를 동일하게 사용하는 것이 아니라 prompt를 적절하게 generate를 시켜줄 수는 없을까?

text description을 요약 모델에 넣어주어 title 혹은 summary를 요약해준다.

summary와 원래의 text description을 모델에 동시에 제공하고 그것을 토대로 학습.

그렇다면 GAN처럼 원래의 loss와 prompt를 넣었을 때의 loss를 비교하고 loss가 최소화되는 방향으로 prompt를 regenerate시키는 방향..?

우선 llama-7b1와 phi-1.5b가 inference는 되는지 확인

In [5]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00


In [18]:
import os
import sys

import numpy
import pandas as pd
import torch
import transformers

import json

from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

# data = load_dataset("codeparrot/apps")

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# print(data)

torch.set_default_device("cuda")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", trust_remote_code=True)
inputs = tokenizer('''```python
    def print_prime(n):
    """
    Print all primes between 1 and n
    """''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)

OutOfMemoryError: ignored

llama-7b는 T4로는 load 자체도 터져버린다.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.6 MB/s eta 0:00:00
